# ASQUIRE DATA LOADING

In [2]:
from tqdm.autonotebook import tqdm
import soundfile as sf
import pandas as pd
import numpy as np
import subprocess
import functools
import librosa
import shutil
import glob
import json
import os

mkdir = lambda p: 0 if os.path.exists(p) else (os.makedirs(p), 1)[1]

/tmp/ipykernel_303106/625706906.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## DATA PATHS

In [10]:
EXP_VER = "v12"
REPORT_FOLDER = f"report_{EXP_VER}"

ASQ_MUSTER_CSV = f"{REPORT_FOLDER}/asquire_corpus_muster.csv"
ASQ_LABELS_CSV = f"{REPORT_FOLDER}/all_asq_aud_anotes.csv"

In [4]:
pd.read_csv(ASQ_MUSTER_CSV)

,index,sub_id,age,gender,height,weight,meta-data--file_path,anot--meta-data--file_path,breath--file_path,anot--breath--file_path,...,uu--file_path,anot--uu--file_path,yee--file_path,anot--yee--file_path,zz--file_path,anot--zz--file_path,sent--file_path,anot--sent--file_path,~noise~--file_path,anot--~noise~--file_path
0,0,qwer-0177a438,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,1,bugsbunny-13f848d8,-,-,-,-,-,-,report_v12/asquire_data/bugsbunny-13f848d8/web...,-,...,report_v12/asquire_data/bugsbunny-13f848d8/web...,-,report_v12/asquire_data/bugsbunny-13f848d8/web...,-,report_v12/asquire_data/bugsbunny-13f848d8/web...,-,report_v12/asquire_data/bugsbunny-13f848d8/web...,-,report_v12/asquire_data/bugsbunny-13f848d8/web...,-
2,2,doll-808ed97a,-,-,-,-,-,-,report_v12/asquire_data/doll-808ed97a/webapp-a...,-,...,report_v12/asquire_data/doll-808ed97a/webapp-a...,-,report_v12/asquire_data/doll-808ed97a/webapp-a...,-,report_v12/asquire_data/doll-808ed97a/webapp-a...,-,report_v12/asquire_data/doll-808ed97a/webapp-a...,-,report_v12/asquire_data/doll-808ed97a/webapp-a...,-
3,3,varun-8f78ac35,-,-,-,-,-,-,report_v12/asquire_data/varun-8f78ac35/webapp-...,-,...,report_v12/asquire_data/varun-8f78ac35/webapp-...,-,report_v12/asquire_data/varun-8f78ac35/webapp-...,-,report_v12/asquire_data/varun-8f78ac35/webapp-...,-,report_v12/asquire_data/varun-8f78ac35/webapp-...,-,report_v12/asquire_data/varun-8f78ac35/webapp-...,-
4,4,murugeswari-d700dff3,-,-,-,-,-,-,report_v12/asquire_data/murugeswari-d700dff3/w...,-,...,report_v12/asquire_data/murugeswari-d700dff3/w...,-,report_v12/asquire_data/murugeswari-d700dff3/w...,-,report_v12/asquire_data/murugeswari-d700dff3/w...,-,report_v12/asquire_data/murugeswari-d700dff3/w...,-,report_v12/asquire_data/murugeswari-d700dff3/w...,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,227,harisk-6f82d610,58,m,175,88,report_v12/asquire_data/harisk-6f82d610/webapp...,report_v12/asquire_data/harisk-6f82d610/webapp...,report_v12/asquire_data/harisk-6f82d610/webapp...,report_v12/asquire_data/harisk-6f82d610/webapp...,...,report_v12/asquire_data/harisk-6f82d610/webapp...,report_v12/asquire_data/harisk-6f82d610/webapp...,-,-,report_v12/asquire_data/harisk-6f82d610/webapp...,report_v12/asquire_data/harisk-6f82d610/webapp...,-,-,-,-
228,228,arhaam-350fc3e6,6,m,114,30,report_v12/asquire_data/arhaam-350fc3e6/webapp...,report_v12/asquire_data/arhaam-350fc3e6/webapp...,report_v12/asquire_data/arhaam-350fc3e6/webapp...,report_v12/asquire_data/arhaam-350fc3e6/webapp...,...,report_v12/asquire_data/arhaam-350fc3e6/webapp...,report_v12/asquire_data/arhaam-350fc3e6/webapp...,-,-,-,-,-,-,-,-
229,229,ravisk-cd186836,60,m,177,83,report_v12/asquire_data/ravisk-cd186836/webapp...,report_v12/asquire_data/ravisk-cd186836/webapp...,report_v12/asquire_data/ravisk-cd186836/webapp...,report_v12/asquire_data/ravisk-cd186836/webapp...,...,report_v12/asquire_data/ravisk-cd186836/webapp...,report_v12/asquire_data/ravisk-cd186836/webapp...,report_v12/asquire_data/ravisk-cd186836/webapp...,report_v12/asquire_data/ravisk-cd186836/webapp...,report_v12/asquire_data/ravisk-cd186836/webapp...,report_v12/asquire_data/ravisk-cd186836/webapp...,-,-,-,-
230,230,srinivas-b89febe6,60,m,172,85,report_v12/asquire_data/srinivas-b89febe6/weba...,report_v12/asquire_data/srinivas-b89febe6/weba...,report_v12/asquire_data/srinivas-b89febe6/weba...,report_v12/asquire_data/srinivas-b89febe6/weba...,...,report_v12/asquire_data/srinivas-b89febe6/weba...,report_v12/asquire_data/srinivas-b89febe6/weba...,report_v12/asquire_data/srinivas-b89febe6/weba...,report_v12/asquire_data/srinivas-b89febe6/weba...,report_v12/asquire_data/srinivas-b89febe6/weba...,report_v12/asquire_data/srinivas-b89febe6/weba...,-,-,-,-


In [9]:
pd.read_csv(ASQ_LABELS_CSV)


,app_code,sub_id,file_class,file_xindex,score,file_format,file_name,file_path,file_match,anot--file_format,...,anot--file_path,age,gender,height,weight,start,end,label,line_number,dur
0,webapp-asquire-mox,bhargavee-70c0073e,oo,8,4.0,wav,webapp-asquire-mox_bhargavee-70c0073e_oo_8_4.wav,report_v12/asquire_data/bhargavee-70c0073e/web...,webapp-asquire-mox_bhargavee-70c0073e_oo_8_4,txt,...,report_v12/asquire_data/bhargavee-70c0073e/web...,21,f,168,48,1.526465,5.969838,oo,1,4.443373
1,webapp-asquire-mox,bhargavee-70c0073e,oo,8,4.0,wav,webapp-asquire-mox_bhargavee-70c0073e_oo_8_4.wav,report_v12/asquire_data/bhargavee-70c0073e/web...,webapp-asquire-mox_bhargavee-70c0073e_oo_8_4,txt,...,report_v12/asquire_data/bhargavee-70c0073e/web...,21,f,168,48,6.861535,11.486270,oo,2,4.624735
2,webapp-asquire-mox,bhargavee-70c0073e,oo,8,4.0,wav,webapp-asquire-mox_bhargavee-70c0073e_oo_8_4.wav,report_v12/asquire_data/bhargavee-70c0073e/web...,webapp-asquire-mox_bhargavee-70c0073e_oo_8_4,txt,...,report_v12/asquire_data/bhargavee-70c0073e/web...,21,f,168,48,12.529103,16.020324,oo,3,3.491221
3,webapp-asquire-mox,bhargavee-70c0073e,yee,9,3.0,wav,webapp-asquire-mox_bhargavee-70c0073e_yee_9_3.wav,report_v12/asquire_data/bhargavee-70c0073e/web...,webapp-asquire-mox_bhargavee-70c0073e_yee_9_3,txt,...,report_v12/asquire_data/bhargavee-70c0073e/web...,21,f,168,48,0.804811,7.611211,yy,1,6.806400
4,webapp-asquire-mox,bhargavee-70c0073e,yee,9,3.0,wav,webapp-asquire-mox_bhargavee-70c0073e_yee_9_3.wav,report_v12/asquire_data/bhargavee-70c0073e/web...,webapp-asquire-mox_bhargavee-70c0073e_yee_9_3,txt,...,report_v12/asquire_data/bhargavee-70c0073e/web...,21,f,168,48,8.576984,13.428843,yy,2,4.851859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6108,webapp-asquire-mox,aditisatvika-dd09b820,ss,7,9.0,wav,webapp-asquire-mox_aditisatvika-dd09b820_ss_7_...,report_v12/asquire_data/aditisatvika-dd09b820/...,webapp-asquire-mox_aditisatvika-dd09b820_ss_7_9,txt,...,report_v12/asquire_data/aditisatvika-dd09b820/...,16,f,170,40,14.950703,26.865730,ss,2,11.915027
6109,webapp-asquire-mox,aditisatvika-dd09b820,ss,7,9.0,wav,webapp-asquire-mox_aditisatvika-dd09b820_ss_7_...,report_v12/asquire_data/aditisatvika-dd09b820/...,webapp-asquire-mox_aditisatvika-dd09b820_ss_7_9,txt,...,report_v12/asquire_data/aditisatvika-dd09b820/...,16,f,170,40,29.142486,41.436973,ss,3,12.294487
6110,webapp-asquire-mox,aditisatvika-dd09b820,ee,3,9.0,wav,webapp-asquire-mox_aditisatvika-dd09b820_ee_3_...,report_v12/asquire_data/aditisatvika-dd09b820/...,webapp-asquire-mox_aditisatvika-dd09b820_ee_3_9,txt,...,report_v12/asquire_data/aditisatvika-dd09b820/...,16,f,170,40,2.262486,17.569622,ee,1,15.307136
6111,webapp-asquire-mox,aditisatvika-dd09b820,ee,3,9.0,wav,webapp-asquire-mox_aditisatvika-dd09b820_ee_3_...,report_v12/asquire_data/aditisatvika-dd09b820/...,webapp-asquire-mox_aditisatvika-dd09b820_ee_3_9,txt,...,report_v12/asquire_data/aditisatvika-dd09b820/...,16,f,170,40,19.266486,29.129514,ee,2,9.863028


In [ ]:
class DataFolds():

    def __init__(self, annotation_csv: str) -> None:
        
        self.ASQ_ANNOTATION_DF = pd.read_csv(annotation_csv)

    
    def make_subject_dfs(self) -> None:
        """
        Creates a dictionary of subject dataframes.
        """
        self.SUBJECT_DFS = {}
        for subject in self.ASQ_ANNOTATION_DF['subject_id'].unique():
            self.SUBJECT_DFS[subject] = self.ASQ_ANNOTATION_DF[self.ASQ_ANNOTATION_DF['subject_id'] == subject]